In [4]:
"""
Functions for loading SAC format seismic data
"""
import os
import glob
from obspy import read, Stream
import numpy as np
import sys
sys.path.append('..')
from config import RAW_DATA_DIR

Configuration loaded. Project root: /Users/devrev/Downloads/repos/GEOP/Multichannel-Analysis-of-Surface-Waves


In [3]:
def load_geophydog_data(data_dir):
    """
    Load all SAC files from Geophydog directory
    
    Parameters:
    -----------
    data_dir : str
        Path to directory containing SAC files
    
    Returns:
    --------
    stream : obspy.Stream
        Stream object containing all traces
    """
    sac_files = sorted(glob.glob(os.path.join(data_dir, "*.sac")))
    
    if not sac_files:
        raise FileNotFoundError(f"No SAC files found in {data_dir}")
    
    print(f"Loading {len(sac_files)} SAC files...")
    stream = Stream()
    
    for sac_file in sac_files:
        st = read(sac_file)
        stream += st
    
    print(f"Loaded {len(stream)} traces")
    return stream

def get_acquisition_geometry(stream):
    """
    Extract acquisition geometry from stream
    
    Parameters:
    -----------
    stream : obspy.Stream
        Stream containing traces
    
    Returns:
    --------
    geometry : dict
        Dictionary with source_offset, receiver_spacing, distances
    """
    distances = []
    
    for tr in stream:
        if hasattr(tr.stats.sac, 'dist'):
            distances.append(tr.stats.sac.dist)
        elif hasattr(tr.stats, 'distance'):
            distances.append(tr.stats.distance)
    
    if not distances:
        print("Warning: No distance information found in headers")
        return None
    
    distances = np.array(distances)
    
    geometry = {
        'distances': distances,
        'receiver_spacing': np.median(np.diff(sorted(distances))),
        'source_offset': min(distances),
        'n_receivers': len(distances)
    }
    
    return geometry

In [5]:
stream = load_geophydog_data(RAW_DATA_DIR)

Loading 60 SAC files...
Loaded 60 traces


In [6]:
stream

60 Trace(s) in Stream:

.GRN21..Z | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:00:07.997944Z | 512.0 Hz, 4096 samples
...
(58 other traces)
...
.GRN21..Z | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:00:07.997944Z | 512.0 Hz, 4096 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

In [ ]:
stream = load_geophydog_data(RAW_DATA_DIR)
geometry = get_acquisition_geometry(stream)

if geometry:
    print("\n--- Acquisition Geometry ---")
    for key, value in geometry.items():
        print(f"{key}: {value}")